In [38]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import r2_score, mean_squared_error

# Set plot style for professional aesthetics
sns.set_theme(style="whitegrid")

# Cytokine Dose-Response Simulation

This notebook demonstrates a computational approach to understanding cytokine dose-response relationships using machine learning and dimensionality reduction.

## Overview
Thwe analysis follows three key steps:
1. **Visualization (The Manifold)**: Use PCA to explore the biological state space
2. **The Predictor (The AI Model)**: Train a machine learning model to predict gene expression
3. **The Virtual Experiment**: Simulate dose-response curves for unseen conditions

## 1. Load Data

Load the experimental data containing donor information, cytokine doses, marker gene expression, and genome-wide measurements.

**Data Assumptions:**
- `data.csv` contains donor metadata (`Donor_ID`, `Cytokine_Dose`)
- `Marker_Gene_Response` is the target variable
- Additional columns represent genome-wide gene expression measurements

In [39]:
try:
    df = pd.read_csv('data/data.csv')
    print(f"   Shape: {df.shape}")
    print(f"   Columns: {df.columns.tolist()[:5]}...") # Preview first 5 cols
except FileNotFoundError:
    exit()

   Shape: (1000, 9)
   Columns: ['Donor_ID', 'Cytokine_Dose', 'Marker_Gene_Response', 'Housekeeping_Gene', 'Inflammatory_Gene_1']...


## Step A: Visualization (The Manifold)

Explore the biological state space using Principal Component Analysis (PCA). This helps us understand how different cytokine doses affect the overall gene expression landscape.

**Approach:**
- Focus on genome-wide gene expression data (excluding metadata)
- Use PCA to reduce dimensionality to 2D for visualization
- Color points by cytokine dose to identify patterns

In [40]:
print("\n--- Running Step A: PCA Visualization ---")

# For PCA, we focus on genome-wide gene expression (all numeric gene columns)
# excluding metadata to reveal biological state clusters
metadata_cols = ['Donor_ID', 'Cytokine_Dose']
features_for_pca = df.drop(columns=metadata_cols + ['Marker_Gene_Response'], errors='ignore')

# Standardize the data (PCA requires unit variance)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(features_for_pca.select_dtypes(include=np.number))

# Apply PCA
n_components = min(2, X_scaled.shape[1])
pca = PCA(n_components=n_components)
principal_components = pca.fit_transform(X_scaled)

# Create a DataFrame for plotting
if n_components >= 2:
    pca_df = pd.DataFrame(data=principal_components, columns=['PC1', 'PC2'])
else:
    # If only 1 component available, duplicate it for 2D plotting
    pca_df = pd.DataFrame(data=np.column_stack([principal_components[:, 0], principal_components[:, 0]]),
                         columns=['PC1', 'PC2'])

pca_df['Cytokine_Dose'] = df['Cytokine_Dose']
pca_df['Donor_ID'] = df['Donor_ID']

# Plotting the Manifold
plt.figure(figsize=(10, 6))
scatter = sns.scatterplot(
    data=pca_df,
    x='PC1',
    y='PC2',
    hue='Cytokine_Dose',
    palette='viridis',
    s=100,
    alpha=0.8,
    edgecolor='w'
)
plt.title('Step A: Biological State Space (PCA)', fontsize=16)
if len(pca.explained_variance_ratio_) >= 2:
    plt.xlabel(f'PC1 ({pca.explained_variance_ratio_[0]:.1%} variance)', fontsize=12)
    plt.ylabel(f'PC2 ({pca.explained_variance_ratio_[1]:.1%} variance)', fontsize=12)
else:
    plt.xlabel(f'PC1 ({pca.explained_variance_ratio_[0]:.1%} variance)', fontsize=12)
    plt.ylabel('PC1 (duplicated for 2D visualization)', fontsize=12)
plt.legend(title='Cytokine Dose (ng/mL)')
plt.savefig('visualization/biological_state_space_pca.png', dpi=300, bbox_inches='tight')
plt.close()


--- Running Step A: PCA Visualization ---


## Step B: The Predictor (The AI Model)

Train a machine learning model to predict marker gene response based on cytokine dose and donor identity.

**Features:**
- `Cytokine_Dose`: Numerical input (treatment level)
- `Donor_ID`: Categorical input (biological variability)

**Model:** Random Forest Regressor with preprocessing pipeline

In [41]:
print("\n--- Running Step B: Training the Models ---")

# Prepare inputs (X) and target output (y)
X = df[['Cytokine_Dose', 'Donor_ID']]
y = df['Marker_Gene_Response']

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# --- Random Forest Model ---
# Create a Preprocessing Pipeline for RF
# 1. 'passthrough' the numerical Cytokine_Dose
# 2. 'OneHotEncode' the categorical Donor_ID
rf_preprocessor = ColumnTransformer(
    transformers=[
        ('num', 'passthrough', ['Cytokine_Dose']),
        ('cat', OneHotEncoder(handle_unknown='ignore'), ['Donor_ID'])
    ]
)

# Create the Full Pipeline (Preprocessor + Random Forest)
rf_pipeline = Pipeline(steps=[
    ('preprocessor', rf_preprocessor),
    ('regressor', RandomForestRegressor(n_estimators=100, random_state=42))
])

# Train the RF model
rf_pipeline.fit(X_train, y_train)

# Evaluate RF
y_pred_rf = rf_pipeline.predict(X_test)
r2_rf = r2_score(y_test, y_pred_rf)
mse_rf = mean_squared_error(y_test, y_pred_rf)

print(f"   R² Score: {r2_rf:.3f} (1.0 is perfect)")
print(f"   Mean Squared Error: {mse_rf:.3f}")

# --- Support Vector Regression (SVR) Model ---
# Create a Preprocessing Pipeline for SVR
# 1. StandardScaler for numerical Cytokine_Dose (Crucial for SVR/RBF)
# 2. OneHotEncode for categorical Donor_ID
svr_preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), ['Cytokine_Dose']),
        ('cat', OneHotEncoder(handle_unknown='ignore'), ['Donor_ID'])
    ]
)

# Create the Full Pipeline (Preprocessor + SVR)
svr_pipeline = Pipeline(steps=[
    ('preprocessor', svr_preprocessor),
    ('regressor', SVR(kernel='rbf', C=100, gamma=0.1, epsilon=0.1))
])

# Train the SVR model
svr_pipeline.fit(X_train, y_train)

# Evaluate SVR
y_pred_svr = svr_pipeline.predict(X_test)
r2_svr = r2_score(y_test, y_pred_svr)
mse_svr = mean_squared_error(y_test, y_pred_svr)

print(f"   R² Score: {r2_svr:.3f}")
print(f"   Mean Squared Error: {mse_svr:.3f}")


--- Running Step B: Training the Models ---
   R² Score: 0.935 (1.0 is perfect)
   Mean Squared Error: 2.314
   R² Score: 0.935
   Mean Squared Error: 2.306


## Step C: The Virtual Experiment (Comparison)

Simulate dose-response curves for unseen experimental conditions using both trained models. This demonstrates how different machine learning approaches can capture biological responses.

**Simulation Approach:**
1. Select a donor from the dataset
2. Generate virtual doses across a continuous range (0-100 ng/mL)
3. Predict marker gene expression using Random Forest and SVR
4. Compare predictions side-by-side with actual experimental data

In [42]:
print("\n--- Running Step C: The Virtual Experiment (Comparison) ---")

# Select a specific donor to simulate
target_donor = df['Donor_ID'].unique()[0]
print(f"   Simulating response for: {target_donor}")

# Generate virtual doses across a continuous range (including unseen doses)
virtual_doses = np.linspace(0, 100, 100)

# 2. Create a dataframe for prediction
virtual_input = pd.DataFrame({
    'Cytokine_Dose': virtual_doses,
    'Donor_ID': [target_donor] * 100
})

# 3. Predict using both trained model pipelines
rf_response = rf_pipeline.predict(virtual_input)
svr_response = svr_pipeline.predict(virtual_input)

# 4. Get the ACTUAL data for this donor to compare
actual_data = df[df['Donor_ID'] == target_donor]

# Plotting the Virtual Experiment - Side by Side Comparison
fig, axes = plt.subplots(1, 2, figsize=(16, 6), sharey=True)

# Plot 1: Random Forest
axes[0].plot(
    virtual_doses, 
    rf_response, 
    color='red', 
    linewidth=3, 
    label='RF Prediction'
)
axes[0].scatter(
    actual_data['Cytokine_Dose'],
    actual_data['Marker_Gene_Response'],
    color='blue',
    s=100,
    label='Actual Lab Data',
    zorder=5
)
axes[0].set_title(f'Random Forest (R²={r2_rf:.2f})', fontsize=16)
axes[0].set_xlabel('Cytokine Dosage (ng/mL)', fontsize=12)
axes[0].set_ylabel('Marker Gene Response', fontsize=12)
axes[0].legend()
axes[0].grid(True, linestyle='--', alpha=0.7)

# Plot 2: Support Vector Regression (SVR)
axes[1].plot(
    virtual_doses, 
    svr_response, 
    color='green', 
    linewidth=3, 
    label='SVR Prediction'
)
axes[1].scatter(
    actual_data['Cytokine_Dose'],
    actual_data['Marker_Gene_Response'],
    color='blue',
    s=100,
    label='Actual Lab Data',
    zorder=5
)
axes[1].set_title(f'SVR (RBF Kernel) (R²={r2_svr:.2f})', fontsize=16)
axes[1].set_xlabel('Cytokine Dosage (ng/mL)', fontsize=12)
# axes[1].set_ylabel('Marker Gene Response') # Shared Y axis label not needed
axes[1].legend()
axes[1].grid(True, linestyle='--', alpha=0.7)

plt.tight_layout()
plt.savefig('visualization/comparison_dose_response_curve.png', dpi=300, bbox_inches='tight')
plt.close()


--- Running Step C: The Virtual Experiment (Comparison) ---
   Simulating response for: Donor_A


## Summary

This analysis demonstrates a complete computational biology workflow:

1. **Data Visualization**: PCA revealed how cytokine doses influence the global gene expression landscape
2. **Predictive Modeling**: A machine learning model was trained to predict marker gene expression from dose and donor information
3. **Virtual Experiments**: The model enables simulation of dose-response relationships for conditions not tested experimentally

**Key Insights:**
- The red curve shows AI-predicted responses across the full dose range
- Blue points represent actual experimental measurements
- This approach can guide experimental design and reduce laboratory costs by predicting optimal dose ranges

**Next Steps:**
- Validate predictions with additional experimental data
- Extend to multi-cytokine combinations
- Incorporate time-course dynamics